# Testing Defacing PET images only

There are a few quirks to this code and this notebook is set up to help a user deal with them. For starters, the user first needs to define where each of their datasets are located in the `dataset_dictionary`, this provides common names for later comparison.

```json
dataset_dictionary = {
    "tracer1": "/full/path/to/tracer1dataset/",
    "tracer2": "/full/path/to/tracer2dataset/",
    "tracer3": "/full/path/to/tracer3dataset/"
}
```

Using the datasets above, this notebook will take the user through the defacing of each tracer using 4 different methods:

1. ~~Using the subject's MR image to register w/ and deface it's PET image.~~

2. Using `petdeface/data/sub-01/ses-baseline/anat/sub-01_T1w.nii.gz` as a template to deface each subjects PET images.

3. Using `petdeface/data/sub-mni305/anat/sub-mni305_T1w.nii.gz` ...

4. Creating a "T1" image by averaging the subjects PET image and using that instead to deface.

At the end of this pipeline the user will have ~~16~~12 different datasets.

In [2]:
import logging
from tqdm.notebook import tqdm
import sys
from datetime import datetime
import os

# Set up logging
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, f"petdeface_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler(sys.stdout)
    ]
)

In [3]:
dataset_dictionary = {
    "PS13": "/home/galassiae/Projects/petdeface_no_anat_data/20M0157_MDD_COX1_COX2_BIDS",
    "MC1": "/home/galassiae/Projects/petdeface_no_anat_data/COX-2_MC1_BIDS",
    "TSPO": "/home/galassiae/Projects/petdeface_no_anat_data/TSPO_Brain"
}

# we'll also need to specify were we want to keep our results

import os
import pathlib

home = pathlib.Path.home()
results_dir = home / "petdeface_jupypter_book_results"
results_dir.mkdir(exist_ok=True)


Next we build, install, and import the code we need.

In [4]:
%cd ..
%pip install -e .
# make sure we can import petdeface
from petdeface.petdeface import PetDeface

/home/galassiae/Projects/petdeface


/home/galassiae/.cache/pypoetry/virtualenvs/petdeface-LqsB4PTs-py3.11/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Obtaining file:///home/galassiae/Projects/petdeface
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for petdeface (pyproject.toml) ... done
  Created wheel for petdeface: filename=petdeface-0.2.2-py3-none-any.whl size=5842 sha256=3021cd28e454caceabd10a5b86f4d5d38b0455ea6b0f64fb7a10d71b0ff6cd7a
  Stored in directory: /home/tmp/pip-ephem-wheel-cache-wo6pofrx/wheels/37/dd/a2/64cedb9880fd17ddc66a522d2179134857fd4f8e010f78eb55
Successfully built petdeface
  Attempting uninstall: petdeface
    Found existing installation: petdeface 0.2.2
    Uninstalling petdeface-0.2.2:
      Successfully uninstalled petdeface-0.2.2

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
# We're going to make a mess if we don't use tempfile
from tempfile import TemporaryDirectory
import shutil
import os
#from tqdm.notebook import tqdm
import sys
from datetime import datetime
import logging
import logging
from tqdm.notebook import tqdm
import sys
from datetime import datetime

log_dir = results_dir / "logs"
os.makedirs(log_dir, exist_ok=True)

for tracer, dataset in tqdm(dataset_dictionary.items(), desc=f"Processing Datasets"):
    #methods = tqdm(['t1', 'mni', 'pet'], desc=f"Processing Tracer {tracer}")
    methods = tqdm(['mni'], desc=f"Processing Tracer {tracer}")
    for method in methods:
        methods.set_description(f"Processing Tracer {tracer} using method {method}")
        tmpdir = TemporaryDirectory()
        faced_dir = pathlib.Path(os.path.join(tmpdir.name, tracer + f"_{method}"))
        defaced_dir = pathlib.Path(results_dir) / f"{tracer}_{method}_defaced"
        shutil.copytree(dataset, faced_dir, dirs_exist_ok=True)
        if not defaced_dir.exists():
            petdeface = PetDeface(
                bids_dir=faced_dir,
                output_dir=faced_dir,
                anat_only=False,
                n_procs=4,
                skip_bids_validator=True,
                remove_existing=False,
                placement="adjacent",
                preview_pics=False,
                use_template_anat=method
                )
            # now we run petdeface on each dataset
            petdeface.run();
            # then once it's finished running we copy over the defaced data to our results directory
            shutil.copytree(
                str(faced_dir) + '_defaced', 
                results_dir / f"{tracer}_{method}_defaced",
                dirs_exist_ok=True)
        #shutil.copytree(os.path.join(tmpdir, tracer + "_defaced"), results_dir / tracer + "_defaced")
            logging.info(f"Data can be found at {results_dir}")
            logging.info(f"finished processing for {tracer}:{method}")
        else:
            logging.info(f"Data for method {method} and tracer {tracer} already exists at "
                         f"{defaced_dir}. Moving onto next method/tracer.")



Processing Datasets:   0%|          | 0/3 [00:00<?, ?it/s]

Processing Tracer PS13:   0%|          | 0/1 [00:00<?, ?it/s]

2025-05-01 17:49:27,387 - INFO - Data for method mni and tracer PS13 already exists at /home/galassiae/petdeface_jupypter_book_results/PS13_mni_defaced. Moving onto next method/tracer.


Processing Tracer MC1:   0%|          | 0/1 [00:00<?, ?it/s]

Copied mni anatomical image to /home/tmp/tmpsdoku9ov/MC1_mni/sub-04/anat/sub-04_T1w.nii.gz
Copied mni anatomical metadata to /home/tmp/tmpsdoku9ov/MC1_mni/sub-04/anat/sub-04_T1w.json
Copied mni anatomical image to /home/tmp/tmpsdoku9ov/MC1_mni/sub-04/anat/sub-04_T1w.nii.gz
Copied mni anatomical metadata to /home/tmp/tmpsdoku9ov/MC1_mni/sub-04/anat/sub-04_T1w.json
Copied mni anatomical image to /home/tmp/tmpsdoku9ov/MC1_mni/sub-05/anat/sub-05_T1w.nii.gz
Copied mni anatomical metadata to /home/tmp/tmpsdoku9ov/MC1_mni/sub-05/anat/sub-05_T1w.json
Copied mni anatomical image to /home/tmp/tmpsdoku9ov/MC1_mni/sub-05/anat/sub-05_T1w.nii.gz
Copied mni anatomical metadata to /home/tmp/tmpsdoku9ov/MC1_mni/sub-05/anat/sub-05_T1w.json
Copied mni anatomical image to /home/tmp/tmpsdoku9ov/MC1_mni/sub-06/anat/sub-06_T1w.nii.gz
Copied mni anatomical metadata to /home/tmp/tmpsdoku9ov/MC1_mni/sub-06/anat/sub-06_T1w.json
Copied mni anatomical image to /home/tmp/tmpsdoku9ov/MC1_mni/sub-06/anat/sub-06_T1w.n

Processing Tracer TSPO:   0%|          | 0/1 [00:00<?, ?it/s]

Copied mni anatomical image to /home/tmp/tmp6jd56d78/TSPO_mni/sub-sf06/anat/sub-sf06_T1w.nii.gz
Copied mni anatomical metadata to /home/tmp/tmp6jd56d78/TSPO_mni/sub-sf06/anat/sub-sf06_T1w.json
Copied mni anatomical image to /home/tmp/tmp6jd56d78/TSPO_mni/sub-sf07/anat/sub-sf07_T1w.nii.gz
Copied mni anatomical metadata to /home/tmp/tmp6jd56d78/TSPO_mni/sub-sf07/anat/sub-sf07_T1w.json
Copied mni anatomical image to /home/tmp/tmp6jd56d78/TSPO_mni/sub-sf08/anat/sub-sf08_T1w.nii.gz
Copied mni anatomical metadata to /home/tmp/tmp6jd56d78/TSPO_mni/sub-sf08/anat/sub-sf08_T1w.json
250501-18:08:45,611 nipype.workflow INFO:
	 Generated workflow graph: /home/tmp/tmp6jd56d78/TSPO_mni/petdeface_wf/petdeface.png (graph2use=colored, simple_form=True).
2025-05-01 18:08:45,611 - INFO - Generated workflow graph: /home/tmp/tmp6jd56d78/TSPO_mni/petdeface_wf/petdeface.png (graph2use=colored, simple_form=True).
250501-18:08:45,618 nipype.workflow INFO:
	 Workflow petdeface_wf settings: ['check', 'execution',

In [8]:
# next we're going to gather the outputs for a single subject and open them up in freeview
folders_in_results_dir = [os.path.join(results_dir, dir) for dir in os.listdir(results_dir) if 'defaced' in 

['/home/galassiae/petdeface_jupypter_book_results/PS13_t1_defaced', '/home/galassiae/petdeface_jupypter_book_results/PS13_pet_defaced', '/home/galassiae/petdeface_jupypter_book_results/MC1_t1_defaced', '/home/galassiae/petdeface_jupypter_book_results/MC1_pet_defaced', '/home/galassiae/petdeface_jupypter_book_results/TSPO_t1_defaced', '/home/galassiae/petdeface_jupypter_book_results/TSPO_pet_defaced', '/home/galassiae/petdeface_jupypter_book_results/PS13_mni_defaced']


Default to using 12 dof for new methods and add to cli

Update dof to be 12 forthis

# Show Images
## Tracers
## Different Subjects

# Show Motion Correction

